# HW1 布尔查询之BSBI与索引压缩

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [54]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [55]:
"""import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()"""

"import urllib.request\nimport zipfile\n\ndata_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'\ndata_dir = 'pa1-data'\nurllib.request.urlretrieve(data_url, data_dir+'.zip')\nzip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')\nzip_ref.extractall()\nzip_ref.close()"

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [56]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [57]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [58]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [59]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [60]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [61]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap""" 
        return len(self.id_to_str)
    
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        try:
            return self.id_to_str[i]
        except IndexError:
            raise IndexError(f"ID {i} 超出范围")
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s in self.str_to_id:
            return self.str_to_id[s]
        else:
            new_id = len(self.id_to_str)
            self.id_to_str.append(s)
            self.str_to_id[s] = new_id
        return new_id
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError("类型不合法！")

确保代码能通过以下简单测试样例

In [62]:
# 测试 IdMap
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [63]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [64]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [65]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [66]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = 'index'  # 这里填入你的索引名称
        self.directory = output_dir  # 设定directory属性
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir,  # type: ignore
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir,  # type: ignore
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id,  # type: ignore
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [67]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        term_doc_pairs = []
        block_dir_path = os.path.join(self.data_dir, block_dir_relative)
        for root, dirs, files in os.walk(block_dir_path):
            for file in files:
                with open(os.path.join(root, file), 'r') as f:
                    doc_id = self.doc_id_map[file]
                    for line in f:
                        for term in line.strip().split():
                            term_id = self.term_id_map[term]
                            term_doc_pairs.append((term_id, doc_id))
        return term_doc_pairs
        ### End your code

观察函数在测试数据上是否正常运行

In [68]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [69]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (4, 1),
 (8, 1)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [70]:
### Begin your code
with open('toy-data/1/bye.txt', 'r') as f:
    print(f.read())
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')
###所以到底要什么样的测试样例？这个算吗？算吧
### End your code

good to see you , bye
see you later ?



[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (4, 1),
 (8, 1)]

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [71]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        # 使用 self.postings_encoding 的 encode 方法，将 postings_list（文档 ID 的列表）编码为字节流 postings_bytes
        postings_bytes = self.postings_encoding.encode(postings_list)

        # 得到当前位置
        # 方法比较原始，不过好在能用
        start_position = self.index_file.tell()
        number_of_postings = len(postings_list)
        length_in_bytes = len(postings_bytes)

        # 更新，更新
        self.terms.append(term)
        self.postings_dict[term] = (start_position, number_of_postings, length_in_bytes)

        # 把二进制文件加进去
        self.index_file.write(postings_bytes)
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [72]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"
    #这个代码段没反应就是没问题

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [73]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code

        from collections import defaultdict ##defaultdict即默认字典，为了处理带有默认值的字典结构

        # 存储倒排列表
        postings = defaultdict(list)
        for term_id, doc_id in td_pairs:
            postings[term_id].append(doc_id)
        
        for term_id, postings_list in postings.items():
            index.append(term_id, postings_list)
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [74]:
### Begin your code
td_pairs = [
    (1, 2), (1, 3), (1, 4),
    (2, 3), (2, 4), (2, 5)  
]

with InvertedIndexWriter('test_invert_write', directory='tmp/', postings_encoding=UncompressedPostings) as index:
    
    bsbi_instance = BSBIIndex(data_dir='', output_dir='', postings_encoding=UncompressedPostings)
    bsbi_instance.invert_write(td_pairs, index)
    index.index_file.seek(0)
    assert index.terms == [1, 2], "terms sequence incorrect"
    expected_postings_dict = {
        1: (0, 3, len(UncompressedPostings.encode([2, 3, 4]))),
        2: (len(UncompressedPostings.encode([2, 3, 4])), 3, len(UncompressedPostings.encode([3, 4, 5])))
    }
    assert index.postings_dict == expected_postings_dict, "postings_dict incorrect"
    
    # 读取并解码索引文件的内容
    index_contents = index.index_file.read()
    decoded_postings = UncompressedPostings.decode(index_contents)
    expected_postings = [2, 3, 4, 3, 4, 5]
    assert decoded_postings == expected_postings, "postings on disk incorrect"
    
print("通过")
### End your code

通过


### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [75]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.current_term_index = 0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        if self.current_term_index >= len(self.terms):
            raise StopIteration
        else:
            term = self.terms[self.current_term_index]
            start_position, num_postings, length_in_bytes = self.postings_dict[term]
            self.index_file.seek(start_position)
            postings_bytes = self.index_file.read(length_in_bytes)
            postings_list = self.postings_encoding.decode(postings_bytes)
            self.current_term_index += 1
            return term, postings_list
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [76]:
### Begin your code
import shutil#删除用的

# 创建临时目录
index_directory = 'tmp/'
if not os.path.exists(index_directory):
    os.makedirs(index_directory)
test_terms_postings = {
    1: [2, 3, 4],
    2: [3, 4, 5],
    3: [5, 6, 7]
}#我不客气了

index_name = 'test_index'

# 写入索引
with InvertedIndexWriter(index_name, directory=index_directory, postings_encoding=UncompressedPostings) as writer:
    for term, postings_list in test_terms_postings.items():
        writer.append(term, postings_list)

# 迭代遍历索引
with InvertedIndexIterator(index_name, directory=index_directory, postings_encoding=UncompressedPostings) as iterator:
    for term, postings_list in iterator:
        print(f"Term: {term}, Postings List: {postings_list}")

# 删除临时目录和文件
shutil.rmtree(index_directory)
### End your code

Term: 1, Postings List: [2, 3, 4]
Term: 2, Postings List: [3, 4, 5]
Term: 3, Postings List: [5, 6, 7]


> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [77]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [78]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
    
    ### Begin your code 
        ## 注意缩进，这个提示的缩进太误导人了，我说怎么全报错
        import heapq
        import itertools

        merged_items = heapq.merge(*indices, key=lambda x: x[0])

        for term, group in itertools.groupby(merged_items, key=lambda x: x[0]):
            postings_lists = [postings_list for _, postings_list in group]
            merged_postings_iter = heapq.merge(*postings_lists)
            merged_postings = [docID for docID, _ in itertools.groupby(merged_postings_iter)]
            merged_index.append(term, merged_postings)
    ### End your code

首先确保它在测试数据上正常运行

In [79]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

接下来对整个数据集构建索引

In [80]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [81]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [82]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)

    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        if term not in self.terms:
            raise KeyError(f"Term '{term}' not found in index.")

        term_id = self.terms.index(term)
        start_position, number_of_postings, length_in_bytes = self.postings_dict[term_id]
        self.index_file.seek(start_position)
        byte_data = self.index_file.read(length_in_bytes)
        postings_list = self.postings_encoding.decode(byte_data)
        return postings_list


写一些测试样例检测`_get_postings_list`的实现

In [83]:
### Begin your code
def test_get_postings_list():
    index_name = 'index_0'
    directory = 'toy_output_dir'  # 指向包含索引和元数据的目录
    
    with InvertedIndexMapper(index_name, directory=directory) as index:
        print(f"Available terms in index: {index.terms}")

        # 查询存在
        term1 = 'hello'
        if term1 in index.terms:
            postings1 = index[term1]
            print(f"Postings for '{term1}': {postings1}")  # 预期输出相关文档ID

        # 查询不存在
        term2 = 'nonexistent'
        try:
            postings2 = index[term2]
            print(f"Postings for '{term2}': {postings2}")  # 预期抛出KeyError
        except KeyError as e:
            print(e)

        # 查询另一个term
        term3 = 'fine'
        if term3 in index.terms:
            postings3 = index[term3]
            print(f"Postings for '{term3}': {postings3}")  # 预期输出相关文档ID

# 运行测试
test_get_postings_list()
### End your code

Available terms in index: [0, 1, 2, 3, 4, 5, 6, 7, 8]
"Term 'nonexistent' not found in index."


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [84]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    ,
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    intersection = []
    i, j = 0, 0

    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            i += 1
        elif list1[i] > list2[j]:
            j += 1
        else:
            intersection.append(list1[i])
            i += 1
            j += 1
    return intersection
    ### End your code

简单的测试样例

In [85]:
### Begin your code
### 不测了，这能有问题我吃
### End your code

现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [86]:
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        tokens = query.strip().split()
        if not tokens:
            return []
        
        result = None
        with InvertedIndexMapper(self.index_name, directory=self.output_dir, postings_encoding=self.postings_encoding) as index:
            for token in tokens:
                try:
                    term_id = self.term_id_map[token]
                except KeyError:
                    return []  # Token not found, return empty list
                
                postings_list = index._get_postings_list(term_id)
                if result is None:
                    result = set(postings_list)  
                else:
                    result.intersection_update(postings_list)  
        
        return sorted(result) if result else []  

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [87]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

[]

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [88]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [89]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            
            # 打印详细的结果
            print(f"Query: {query.strip()}")
            print("My Results:")
            print(my_results)
            print("Reference Results:")
            print(reference_results)
            print("In My Results but not in Reference Results:")
            print(set(my_results) - set(reference_results))
            print("In Reference Results but not in My Results:")
            print(set(reference_results) - set(my_results))
            
            assert my_results == reference_results, "Results DO NOT match for query: " + query.strip()
        print("Results match for query:", query.strip())

Query: we are
My Results:
[]
Reference Results:
['0\\5-sure.stanford.edu_', '0\\50years.stanford.edu_', '0\\a3cservices.stanford.edu_lead_', '0\\aa.stanford.edu_', '0\\aa.stanford.edu_about_control.php', '0\\aa.stanford.edu_admissions_', '0\\aa.stanford.edu_admissions_admittedGrad.php', '0\\aa.stanford.edu_admissions_fellowsAero.php', '0\\aa.stanford.edu_events_50thAnniversary_media_Ying.pdf', '0\\aa.stanford.edu_faculty_facultySearch.php', '0\\aa.stanford.edu_giving_index.php', '0\\aaalab.stanford.edu_child_development_dev_dominos.html', '0\\aaalab.stanford.edu_child_development_dev_reasoning.html', '0\\aaalab.stanford.edu_face_to_face_interaction_face_agency.html', '0\\aaalab.stanford.edu_imagery_im_complex.html', '0\\aaalab.stanford.edu_teachable_agents_ta_moby.html', '0\\aaastop.stanford.edu_patients_', '0\\aaastop.stanford.edu_patients_our_goal.html', '0\\aaastop.stanford.edu_patients_who_is_eligible.html', '0\\aaastop.stanford.edu_physicians_', '0\\aacf.stanford.edu_', '0\\aagsa.

AssertionError: Results DO NOT match for query: we are

如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [ ]:
set(my_results) - set(reference_results)

set()

In [ ]:
set(reference_results) - set(my_results)

{'5\\scien.stanford.edu_pages_labsite_2005_psych221_projects_05_joanmoh_dsnu.html',
 '9\\www.stanford.edu_group_CIFE_',
 '1\\ed.stanford.edu_news_cuban-students-excel-latin-america',
 '4\\neurosurgery.stanford.edu_referrals_make_referral.html',
 '4\\nmbl.stanford.edu_research_software_index.htm',
 '4\\paesmem.stanford.edu_',
 '0\\biologyprofiles.stanford.edu_profiles_biologyprofiles_researcher_Carol_Boggs',
 '0\\bmir.stanford.edu_publications_view.php_evaluation_of_semantic_based_information_retrieval_methods_in_the_autism_phenotype_domain',
 '9\\www.stanford.edu_group_Zarelab_guinness_',
 '5\\sepwww.stanford.edu_public_docs_sep95_christin1_paper_html_node2.html',
 '3\\ions.stanford.edu_submissions',
 '4\\pangea.stanford.edu_departments_ere_events_ere-winter-seminar-series-2012-overview-upstream-ep-fiscal-regimes',
 '4\\news.stanford.edu_news_2006_december6_mccall-120606.html',
 '2\\hypernews.slac.stanford.edu_HyperNews_geant4_get_hitsdigits_267_1.html',
 '1\\cyberlaw.stanford.edu_node

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [ ]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [ ]:
class CompressedPostings:
    ...
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        def vb_encode_number(number):
            bytes_list = []
            while True:
                bytes_list.insert(0, number % 128)
                if number < 128:
                    break
                number = number // 128
            bytes_list[-1] += 128
            return bytes_list

        def vb_encode(numbers):
            bytes_list = []
            for number in numbers:
                bytes_list.extend(vb_encode_number(number))
            return bytes_list

        gaps = [postings_list[0]] + [postings_list[i] - postings_list[i - 1] for i in range(1, len(postings_list))]
        return bytes(vb_encode(gaps))
        ### End your code

    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        def vb_decode(bytes_list):
            numbers = []
            n = 0
            for byte in bytes_list:
                if byte < 128:
                    n = 128 * n + byte
                else:
                    n = 128 * n + (byte - 128)
                    numbers.append(n)
                    n = 0
            return numbers

        gaps = vb_decode(encoded_postings_list)
        postings_list = [gaps[0]]
        for i in range(1, len(gaps)):
            postings_list.append(postings_list[-1] + gaps[i])
        return postings_list
        ### End your code

首先，如果你实现了额外的辅助函数，写一些测试样例

In [ ]:
### Begin your code

### End your code

我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [ ]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [ ]:
### Begin your code

### End your code

现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [ ]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

像之前一样，用已构造好的查询语句来测试是否正常运行

In [ ]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [ ]:
class ECCompressedPostings:
    ...
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        def gamma_encode_number(number):
            if number == 0:
                return [0]
            binary = bin(number)[2:]
            offset = binary[1:]
            length = len(offset)
            return [0] * length + [1] + [int(bit) for bit in offset]

        def gamma_encode(numbers):
            bits = []
            for number in numbers:
                bits.extend(gamma_encode_number(number))
            return bits

        def bits_to_bytes(bits):
            byte_array = bytearray()
            for i in range(0, len(bits), 8):
                byte = 0
                for bit in bits[i:i+8]:
                    byte = (byte << 1) | bit
                byte_array.append(byte)
            return bytes(byte_array)

        gaps = [postings_list[0]] + [postings_list[i] - postings_list[i - 1] for i in range(1, len(postings_list))]
        return bits_to_bytes(gamma_encode(gaps))
        ### End your code

    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        def bytes_to_bits(byte_array):
            bits = []
            for byte in byte_array:
                for i in range(8):
                    bits.append((byte >> (7 - i)) & 1)
            return bits

        def gamma_decode(bits):
            numbers = []
            i = 0
            while i < len(bits):
                length = 0
                while bits[i] == 0:
                    length += 1
                    i += 1
                i += 1
                offset = 0
                for j in range(length):
                    offset = (offset << 1) | bits[i]
                    i += 1
                number = (1 << length) | offset
                numbers.append(number)
            return numbers

        bits = bytes_to_bits(encoded_postings_list)
        gaps = gamma_decode(bits)
        postings_list = [gaps[0]]
        for i in range(1, len(gaps)):
            postings_list.append(postings_list[-1] + gaps[i])
        return postings_list
        ### End your code

# 作业提交

本次作业用时y约3周，截止日期为9.30（国庆节前）。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到ir24fall@163.com，邮件和文件命名方式均为`学号_姓名_hw1`，如`1811412_戚晓睿_hw1.ipynb`